<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
    <font color=orange>I - 4 </font>
    Sequence Labelling
    
  </div> 

<div style="
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 20px; 
      margin: 10px;">
  a. POS tagging
  </div>


  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

3. Language Modeling

4. <font color=orange>**Sequence Labelling**</font>


### Part II

1. Text Classification

2. Sequence to sequence



### Part III

1. Abstractive Summarization

2. Question Answering

3. Chatbot


</div>

***

<a id="plan"></a>

| | | | |
|------|------|------|------|
| **Content** | [Corpus](#corpus) | [Modules](#modules) | [Model](#model) | 


# Overview

A top-quality Github repository discussing Sequence Labelling is found [here](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling)<br><br><br>


| **Corpus** | **Popular library using the corpus** | **Data availability** | 
|------|------|------|
| [Penn Treebank](https://catalog.ldc.upenn.edu/LDC99T42) | Stanford NLP | Extract in NLTK as detailed [here](https://becominghuman.ai/part-of-speech-tagging-tutorial-with-the-keras-deep-learning-library-d7f93fa05537) | 
| OntoNotes 5 |  |  | 
| Groningen Meaning Bank | | Full data [here](https://gmb.let.rug.nl/data.php) - Extract found [here](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus) |

# Packages

In [1]:
from __future__ import unicode_literals, print_function, division
import sys
import warnings
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 1.4.0
DL device : cuda


In [2]:
path_to_DL4NLP = os.path.dirname(os.getcwd())

In [3]:
sys.path.append(path_to_DL4NLP + '\\lib')

<a id="corpus"></a>

# Corpus

[Back to top](#plan)


### $\bullet$ Groningen Meaning Bank extract

In [4]:
df_GMB_extract = pd.read_csv(path_to_DL4NLP + "\\data\\Groningen Meaning Bank (extract)\\ner.csv", encoding = "ISO-8859-1", error_bad_lines = False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [5]:
df_GMB_extract_add = pd.read_csv(path_to_DL4NLP + "\\data\\Groningen Meaning Bank (extract)\\ner_dataset.csv", encoding = "ISO-8859-1", error_bad_lines = False)

In [6]:
df_GMB_extract.dropna(inplace = True)
df_GMB_extract = df_GMB_extract[['sentence_idx', 'word', 'pos']]
print(df_GMB_extract.shape)
df_GMB_extract.head()

(1050794, 3)


,sentence_idx,word,pos
0,1.0,Thousands,NNS
1,1.0,of,IN
2,1.0,demonstrators,NNS
3,1.0,have,VBP
4,1.0,marched,VBN


In [7]:
df_GMB_extract_add.fillna(method = 'ffill', inplace = True)
df_GMB_extract_add = df_GMB_extract_add[['Sentence #', 'Word', 'POS']]
df_GMB_extract_add.replace('FW', 'NN', inplace = True)
print(df_GMB_extract_add.shape)
df_GMB_extract_add.head()

(1048575, 3)


,Sentence #,Word,POS
0,Sentence: 1,Thousands,NNS
1,Sentence: 1,of,IN
2,Sentence: 1,demonstrators,NNS
3,Sentence: 1,have,VBP
4,Sentence: 1,marched,VBN


In [182]:
POSs = df_GMB_extract['pos'].unique().tolist()
print(len(POSs))
POSs2num = {tag : i for i, tag in enumerate(POSs)}
num2POSs = {i : tag for i, tag in enumerate(POSs)}

41


In [183]:
corpus = df_GMB_extract.groupby("sentence_idx").apply(lambda s: [[w.lower() for w in s["word"].values.tolist()], 
                                                                 [POSs2num[t] for t in s["pos"].values.tolist()]]).tolist()

In [184]:
corpus_add = df_GMB_extract_add.groupby("Sentence #").apply(lambda s: [[w.lower() for w in s["Word"].values.tolist()], 
                                                                       [POSs2num[t] for t in s["POS"].values.tolist()]]).tolist()

In [185]:
corpus_trn, corpus_tst = train_test_split(corpus, test_size = 0.2, random_state = 42)

In [186]:
sentences_trn = [s_l[0] for s_l in corpus_trn if len(s_l[0]) <= 75]
poslabels_trn = [s_l[1] for s_l in corpus_trn if len(s_l[1]) <= 75]

In [187]:
len(corpus_trn)

28141

In [188]:
words = corpus_trn[0][0]
tags  = corpus_trn[0][1]
for word, tag in zip(words, tags) : print('(' + word + ', ' + str(tag)+ ', ' + str(num2POSs[tag]) + ')')

(top-seeded, 10, JJ)
(lleyton, 4, NNP)
(hewitt, 4, NNP)
(of, 1, IN)
(australia, 4, NNP)
(and, 9, CC)
(number-two, 10, JJ)
(dominik, 4, NNP)
(hrbaty, 4, NNP)
(of, 1, IN)
(slovakia, 4, NNP)
(have, 2, VBP)
(advanced, 3, VBN)
(to, 5, TO)
(the, 7, DT)
(second, 10, JJ)
(round, 8, NN)
(of, 1, IN)
(the, 7, DT)
(international, 10, JJ)
(men, 0, NNS)
('s, 18, POS)
(hardcourt, 8, NN)
(tennis, 8, NN)
(championships, 0, NNS)
(in, 1, IN)
(adelaide, 4, NNP)
(,, 21, ,)
(australia, 4, NNP)
(., 11, .)
(top-seeded, 10, JJ)
(lleyton, 4, NNP)
(hewitt, 4, NNP)
(of, 1, IN)
(australia, 4, NNP)
(and, 9, CC)
(number-two, 10, JJ)
(dominik, 4, NNP)
(hrbaty, 4, NNP)
(of, 1, IN)
(slovakia, 4, NNP)
(have, 2, VBP)
(advanced, 3, VBN)
(to, 5, TO)
(the, 7, DT)
(second, 10, JJ)
(round, 8, NN)
(of, 1, IN)
(the, 7, DT)
(international, 10, JJ)
(men, 0, NNS)
('s, 18, POS)
(hardcourt, 8, NN)
(tennis, 8, NN)
(championships, 0, NNS)
(in, 1, IN)
(adelaide, 4, NNP)
(,, 21, ,)
(australia, 4, NNP)
(., 11, .)


<a id="modules"></a>

# 1 Modules

### 1.1 Word Embedding module

[Back to top](#plan)

All details on Word Embedding modules and their pre-training are found in **Part I - 1**. We consider here a FastText model trained following the Skip-Gram training objective.

#### $\bullet$ FastText model

In [12]:
from libDL4NLP.models.Word_Embedding import Word2Vec as myWord2Vec
from libDL4NLP.models.Word_Embedding import Word2VecConnector
from libDL4NLP.utils.Lang import Lang

In [13]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath, get_tmpfile

In [191]:
word2vec = FastText(size = 100, 
                    window = 5, 
                    min_count = 1, 
                    negative = 20,
                    sg = 1)

In [193]:
word2vec.build_vocab(sentences_trn)
len(word2vec.wv.vocab)

24652

In [194]:
word2vec.train(sentences_trn,
               epochs = 20,
               total_examples = word2vec.corpus_count)

### 1.2 Contextualization module

[Back to top](#plan)


In [27]:
from libDL4NLP.modules import RecurrentEncoder

<a id="model"></a>

# 2 Part Of Speech Tagging Model

[Back to top](#plan)


In [154]:
class SequenceTagger(nn.Module) :
    def __init__(self, device, tokenizer, word2vec, 
                 hidden_dim = 100, 
                 n_layers = 1, 
                 n_class = 2,
                 dropout = 0,
                 class_weights = None, 
                 optimizer = optim.SGD
                 ):
        super(SequenceTagger, self).__init__()
        
        # embedding
        self.tokenizer = tokenizer
        self.word2vec  = word2vec
        self.context   = RecurrentEncoder(self.word2vec.output_dim, hidden_dim, n_layers, dropout, bidirectional = True)
        self.out       = nn.Linear(self.context.output_dim, n_class)
        self.act       = F.softmax
        self.n_class   = n_class
        
        # optimizer
        self.ignore_index_in  = self.word2vec.lang.getIndex('PADDING_WORD')
        self.ignore_index_out = n_class
        self.criterion = nn.NLLLoss(size_average = False, 
                                    ignore_index = self.ignore_index_out, 
                                    weight = class_weights)
        self.optimizer = optimizer
        
        # load to device
        self.device = device
        self.to(device)
        
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def predict_proba(self, words):
        embeddings = self.word2vec.twin(words, self.device) # dim = (1, input_length, hidden_dim)
        hiddens, _ = self.context(embeddings)               # dim = (1, input_length, hidden_dim)
        probs      = self.act(self.out(hiddens), dim = 2)   # dim = (1, input_length, n_class)
        return probs

    # main method
    def forward(self, sentence = '.', color = '\033[94m'):
        def addColor(w1, w2, color) : return color + w2 + '\033[0m' if w1 != w2 else w2
        words  = self.tokenizer(sentence)
        probs  = self.predict_proba(words).squeeze(0) # dim = (input_length,  n_class)
        inds   = [probs[i].data.topk(1)[1].item() for i in range(probs.size(0))]
        return [(w, i) for w, i in zip(words, inds)]

    # load data
    def generatePackedSentences(self, 
                                sentences, 
                                batch_size = 32, 
                                mask_ratio = 0,
                                seed = 42) :
        def maskInput(index, b) :
            if   b and random.random() > 0.25 : return self.word2vec.lang.getIndex('UNK')
            elif b and random.random() > 0.10 : return random.choice(list(self.word2vec.twin.lang.word2index.values()))
            else                              : return index
            
        def sentence2indices(words) :
            # convert to indices
            inds = [self.word2vec.lang.getIndex(w) for w in words]
            inds = [i for i in inds if i is not None]
            # apply mask
            mask = [m for m, i in enumerate(inds) if i != self.word2vec.lang.getIndex('UNK')]
            mask = random.sample(mask, k = int(mask_ratio*(len(mask) +1)))
            inds = [maskInput(i, m in mask) for m, i in enumerate(inds)]
            return inds
        
        random.seed(seed)
        sentences.sort(key = lambda s: len(s[0]), reverse = True)
        packed_data = []
        for i in range(0, len(sentences), batch_size) :
            pack = [[sentence2indices(s[0]), s[1]] for s in sentences[i:i + batch_size]]
            pack.sort(key = lambda p : len(p[0]), reverse = True)
            pack0 = [p[0] for p in pack] 
            pack0 = list(itertools.zip_longest(*pack0, fillvalue = self.ignore_index_in))
            pack0 = Variable(torch.LongTensor(pack0).transpose(0, 1)) # size (batch_size, max_length)
            lengths = torch.tensor([len(p[0]) for p in pack])         # size (batch_size)
            pack1 = [p[1] for p in pack]                              # size (batch_size, max_length)
            pack1 = list(itertools.zip_longest(*pack1, fillvalue = self.ignore_index_out))
            pack1 = Variable(torch.LongTensor(pack1).transpose(0, 1)) # size (batch_size, max_length) 
            packed_data.append([[pack0, lengths], pack1])
        return packed_data

    # compute model perf
    def compute_accuracy(self, sentences, batch_size = 32) :
        def computeLogProbs(batch) :
            torch.cuda.empty_cache()
            embeddings = self.word2vec.embedding(batch[0].to(self.device))
            hiddens,_  = self.context(embeddings, lengths = batch[1].to(self.device)) # dim = (batch_size, input_length, hidden_dim)
            log_probs  = F.log_softmax(self.out(hiddens), dim = 2)                    # dim = (batch_size, input_length, lang_size)
            return log_probs

        def computeSuccess(log_probs, targets) :
            total   = np.sum(targets.data.cpu().numpy() != self.ignore_index_out)
            success = sum([self.ignore_index_out != targets[i, j].item() == log_probs[i, :, j].data.topk(1)[1].item() \
                           for i in range(targets.size(0)) \
                           for j in range(targets.size(1)) ])
            return success, total
        
        # --- main ----
        self.eval()
        batches = self.generatePackedSentences(sentences, batch_size)
        score, total = 0, 0
        for batch, targets in batches :
            log_probs = computeLogProbs(batch).transpose(1, 2) # dim = (batch_size, lang_size, input_length)
            targets = targets.to(self.device)                  # dim = (batch_size, input_length)
            s, t = computeSuccess(log_probs, targets)
            score += s
            total += t
        return score * 100 / total
    
    # fit model
    def fit(self, batches, 
            iters = None, epochs = None, lr = 0.025, random_state = 42, 
            print_every = 10, compute_accuracy = True):
        """Performs training over a given dataset and along a specified amount of loops"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0
        
        def computeLogProbs(batch) :
            embeddings = self.word2vec.embedding(batch[0].to(self.device))
            hiddens,_  = self.context(embeddings, lengths = batch[1].to(self.device)) # dim = (batch_size, input_length, hidden_dim)
            log_probs  = F.log_softmax(self.out(hiddens), dim = 2)                    # dim = (batch_size, input_length, lang_size)
            return log_probs

        def computeAccuracy(log_probs, targets) :
            total   = np.sum(targets.data.cpu().numpy() != self.ignore_index_out)
            success = sum([self.ignore_index_out != targets[i, j].item() == log_probs[i, :, j].data.topk(1)[1].item() \
                           for i in range(targets.size(0)) \
                           for j in range(targets.size(1)) ])
            return  success * 100 / total

        def trainLoop(batch, optimizer, compute_accuracy = True):
            """Performs a training loop, with forward pass, backward pass and weight update."""
            torch.cuda.empty_cache()
            optimizer.zero_grad()
            self.zero_grad()
            log_probs  = computeLogProbs(batch[0]).transpose(1, 2) # dim = (batch_size, lang_size, input_length)
            targets    = batch[1].to(self.device)                  # dim = (batch_size, input_length)
            loss       = self.criterion(log_probs, targets)
            loss.backward()
            optimizer.step()
            if compute_accuracy :
                accuracy = computeAccuracy(log_probs, targets)
            else : 
                accuracy = 0
            error = float(loss.item() / np.sum(targets.data.cpu().numpy() != self.ignore_index_out))
            return error, accuracy
        
        # --- main ---
        self.train()
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_acc  = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                batch = random.choice(batches)
                loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                tot_loss += loss
                tot_acc += acc      
                if iter % print_every == 0 : 
                    tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(batches) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(batches)
                for batch in batches :
                    loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                    tot_loss += loss
                    tot_acc += acc 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        return

#### $\bullet$ POSTagger Training

In [285]:
pos_tagger = SequenceTagger(device = torch.device('cpu'), # device
                            tokenizer = lambda s : s.split(' '),
                            word2vec = Word2VecConnector(word2vec),
                            hidden_dim = 125, 
                            n_layers = 2, 
                            n_class = 41,
                            dropout = 0.15,
                            optimizer = optim.AdamW)

pos_tagger.nbParametres()

458166

In [286]:
batches = pos_tagger.generatePackedSentences(corpus_trn, 
                                             batch_size = 16,
                                             mask_ratio = 0.15,
                                             seed = 42)
batches+= pos_tagger.generatePackedSentences(corpus_trn, 
                                             batch_size = 16,
                                             mask_ratio = 0.15,
                                             seed = 4242)
batches+= pos_tagger.generatePackedSentences(corpus_trn, 
                                             batch_size = 16,
                                             mask_ratio = 0.15,
                                             seed = 1331)
len(batches)

5277

In [287]:
pos_tagger.fit(batches, epochs = 1, lr = 0.001, print_every = 50)

epoch 1
0m 14s (- 25m 43s) (50 0%) loss : 2.825  accuracy : 21.6 %
0m 26s (- 22m 52s) (100 1%) loss : 1.676  accuracy : 54.0 %
0m 37s (- 21m 12s) (150 2%) loss : 1.037  accuracy : 72.3 %
0m 47s (- 20m 4s) (200 3%) loss : 0.886  accuracy : 75.2 %
0m 58s (- 19m 32s) (250 4%) loss : 0.806  accuracy : 77.6 %
1m 8s (- 18m 52s) (300 5%) loss : 0.751  accuracy : 78.6 %
1m 19s (- 18m 40s) (350 6%) loss : 0.703  accuracy : 79.9 %
1m 29s (- 18m 11s) (400 7%) loss : 0.694  accuracy : 79.6 %
1m 38s (- 17m 39s) (450 8%) loss : 0.666  accuracy : 80.7 %
1m 46s (- 16m 56s) (500 9%) loss : 0.615  accuracy : 81.7 %
1m 56s (- 16m 40s) (550 10%) loss : 0.637  accuracy : 80.9 %
2m 7s (- 16m 31s) (600 11%) loss : 0.615  accuracy : 81.8 %
2m 17s (- 16m 17s) (650 12%) loss : 0.618  accuracy : 81.5 %
2m 26s (- 15m 57s) (700 13%) loss : 0.597  accuracy : 81.9 %
2m 37s (- 15m 49s) (750 14%) loss : 0.604  accuracy : 81.7 %
2m 48s (- 15m 44s) (800 15%) loss : 0.568  accuracy : 82.4 %
2m 57s (- 15m 26s) (850 16%) l

In [290]:
pos_tagger.fit(batches, epochs = 1, lr = 0.00025, print_every = 50)

epoch 1
0m 10s (- 18m 11s) (50 0%) loss : 0.355  accuracy : 89.2 %
0m 24s (- 20m 58s) (100 1%) loss : 0.375  accuracy : 88.4 %
0m 36s (- 20m 32s) (150 2%) loss : 0.368  accuracy : 88.8 %
0m 48s (- 20m 27s) (200 3%) loss : 0.369  accuracy : 88.8 %
0m 58s (- 19m 38s) (250 4%) loss : 0.372  accuracy : 88.5 %
1m 8s (- 18m 50s) (300 5%) loss : 0.353  accuracy : 89.0 %
1m 17s (- 18m 16s) (350 6%) loss : 0.353  accuracy : 89.0 %
1m 28s (- 18m 0s) (400 7%) loss : 0.363  accuracy : 88.5 %
1m 39s (- 17m 48s) (450 8%) loss : 0.359  accuracy : 88.6 %
1m 48s (- 17m 15s) (500 9%) loss : 0.362  accuracy : 88.9 %
2m 1s (- 17m 26s) (550 10%) loss : 0.377  accuracy : 88.5 %
2m 10s (- 16m 59s) (600 11%) loss : 0.349  accuracy : 89.0 %
2m 22s (- 16m 52s) (650 12%) loss : 0.376  accuracy : 88.4 %
2m 32s (- 16m 40s) (700 13%) loss : 0.358  accuracy : 88.9 %
2m 42s (- 16m 21s) (750 14%) loss : 0.355  accuracy : 89.0 %
2m 53s (- 16m 13s) (800 15%) loss : 0.358  accuracy : 88.9 %
3m 5s (- 16m 3s) (850 16%) los

In [225]:
# save
#torch.save(pos_tagger.state_dict(), path_to_NLP + '\\saves\\models\\DL4NLP_I4a_pos_tagger.pth')

# load
#pos_tagger.load_state_dict(torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I4a_pos_tagger.pth'))

<All keys matched successfully>

#### $\bullet$ POSTagger Evaluation

In [291]:
pos_tagger.compute_accuracy(corpus_tst, batch_size = 16)

95.84015023260446

In [292]:
pos_tagger.compute_accuracy(corpus_add, batch_size = 16)

96.20570774622702

In [216]:
pos_tagger.eval()
sentence = ' '.join(corpus_tst[11][0]) #'what are you thinking of this'
print(sentence)
print('\n')
print(pos_tagger(sentence))
print('\n')
print([(w, i) for w, i in zip(corpus_tst[11][0], corpus_tst[11][1])])

mexico 's andres valencia told reporters late tuesday his talks with national liberation army ( eln ) leader francisco galan focused on ways to reduce differences between the rebels and the government in order to set up a possible meeting between the two sides in mexico . mexico 's andres valencia told reporters late tuesday his talks with national liberation army ( eln ) leader francisco galan focused on ways to reduce differences between the rebels and the government in order to set up a possible meeting between the two sides in mexico .


[('mexico', 4), ("'s", 18), ('andres', 4), ('valencia', 4), ('told', 12), ('reporters', 0), ('late', 10), ('tuesday', 4), ('his', 23), ('talks', 0), ('with', 1), ('national', 4), ('liberation', 4), ('army', 4), ('(', 35), ('eln', 4), (')', 36), ('leader', 8), ('francisco', 4), ('galan', 4), ('focused', 12), ('on', 1), ('ways', 0), ('to', 5), ('reduce', 6), ('differences', 0), ('between', 1), ('the', 7), ('rebels', 0), ('and', 9), ('the', 7), ('gove